In [35]:
from tensorflow.keras.models import load_model
import joblib
import numpy as np
import os

In [2]:
model = load_model('/content/drive/MyDrive/Colab Notebooks/GEN_AI-Intern/Text-Generation/HuggingFace/ChatBot/ques_to_intend.h5')
encoder = joblib.load('/content/drive/MyDrive/Colab Notebooks/GEN_AI-Intern/Text-Generation/HuggingFace/ChatBot/encoder.pkl')
tokenizer = joblib.load('/content/drive/MyDrive/Colab Notebooks/GEN_AI-Intern/Text-Generation/HuggingFace/ChatBot/tokenizer.pkl')

In [3]:
import json
ans_database = open('/content/drive/MyDrive/Colab Notebooks/GEN_AI-Intern/Text-Generation/ChatBot/Ans_data.txt').read()
ans_database = ans_database.replace("'", "\"")

In [4]:
ans_database = json.loads(ans_database)
print(ans_database)

{'greetings': ['Hello Dear,', 'Hello Good Day', 'Hello My name is Zoya', 'Hello, How can I assist you'], 'need-help': ['Hi I am zoya, I would be happy to help you', 'Hi my name is John, I am here to help you'], 'artificial-intelligence': ['Artificial intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think like humans and mimic their actions.']}


In [5]:
intent = list(ans_database.keys())

Importing the en-to-tamil translator model from hugginface

In [6]:
!pip install transformers --quiet

In [7]:
!pip install sentencepiece --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.2 MB/s eta 0:00:00


In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [9]:
translator_tokenizer = AutoTokenizer.from_pretrained("Hemanth-thunder/english-tamil-mt")
translator_model = AutoModelForSeq2SeqLM.from_pretrained("Hemanth-thunder/english-tamil-mt")

tokenizer_config.json:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/931 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Libraries for Text-To-Speech (TTS)

In [11]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00


In [12]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch
import soundfile as sf
from datasets import load_dataset

Processor, model and vocoder for TTs

In [13]:
tts_processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
tts_model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
tts_vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/585M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/50.7M [00:00<?, ?B/s]

Loading xvector containing speaker's voice characteristics from a dataset


In [14]:
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

Generating validation split:   0%|          | 0/7931 [00:00<?, ? examples/s]

In [18]:
!pip install pydub
import pydub

In [21]:
!pip install playsound
import playsound

  Preparing metadata (setup.py) ... done
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=50b1abcbfca004e9c087a2312f9cb4d86a0117860bed12407cc4b07cf35391b3
  Stored in directory: /root/.cache/pip/wheels/90/89/ed/2d643f4226fc8c7c9156fc28abd8051e2d2c0de37ae51ac45c
Successfully built playsound


In [32]:
!pip install IPython
from IPython.display import Audio, display

In [36]:
while True:
  que = input('You:   ')

  if que == "exit":
    print("Thankyou for your time. It's a gread conversation with you ")
    break

  tok_data = tokenizer.texts_to_sequences(que)

  pad_seq = pad_sequences(tok_data, maxlen = 10)
  pred = model.predict(pad_seq)

  out_max = np.zeros_like(pred)
  for i in range(pred.shape[0]):
    out = np.argmax(pred[i])
    out_max[i][out] = 1

  intend = encoder.inverse_transform(out_max)[0][0]
  print(intend)

  # Fetching the answer with intend
  answer = np.random.choice(ans_database[intend])
  print("Awesome bot:    ", answer)

  # Translating to tamil
  trans_tok = translator_tokenizer(answer, return_tensors = 'pt', padding=True)
  trans_out = translator_model.generate(trans_tok['input_ids'])
  trans_output = translator_tokenizer.decode(trans_out[0], skip_special_tokens = True)

  print('எந்திரா:   ', trans_output)

  # TTS
  tts_input = tts_processor(text=answer, return_tensors='pt')
  tts_speech = tts_model.generate_speech(tts_input['input_ids'], speaker_embeddings, vocoder = tts_vocoder)
  sf.write("speech.wav", tts_speech.numpy(), samplerate=16000)
  # Converting the .wav file to audio
  display(Audio('/content/speech.wav', autoplay=True))
  # Deleting the created audio .wav file
  os.remove('/content/speech.wav')

You:   hi
1/1 [==============================] - 0s 23ms/step
greetings
Awesome bot:     Hello Dear,
எந்திரா:    அன்புள்ளவர்களே,


You:   what help from you
1/1 [==============================] - 0s 25ms/step
greetings
Awesome bot:     Hello, How can I assist you
எந்திரா:    ஹலோ நான் உங்களுக்கு எப்படி உதவ முடியும்?


You:   exit
Thankyou for your time. It's a gread conversation with you 
